In [ ]:
# Uploading all professor modules
import sys
sys.path.append('../GithubModules')
print(sys.version)

In [ ]:
import pandas as pd 
URM_df=pd.read_csv('../input/RecsysDataset/data_train.csv')
ICM_genre_df=pd.read_csv('../input/RecsysDataset/data_ICM_genre.csv')
ICM_subgenre_df=pd.read_csv('../input/RecsysDataset/data_ICM_subgenre.csv')
ICM_channel_df=pd.read_csv('../input/RecsysDataset/data_ICM_channel.csv')
ICM_event_df=pd.read_csv('../input/RecsysDataset/data_ICM_event.csv')
# Extract a list of users who will be present in the final submission
target_users = pd.read_csv('../input/RecsysDataset/data_target_users_test.csv')

In [ ]:
# Just for usability/readability + convert values from float to int 

URM_df.columns = ['userID','itemID','interaction']
ICM_channel_df.columns = ['itemID','featureID','value']
ICM_event_df.columns = ['itemID','featureID','value']
ICM_genre_df.columns = ['itemID','featureID','value']
ICM_subgenre_df.columns = ['itemID','featureID','value']
URM_df['interaction'] = URM_df['interaction'].astype(int)
ICM_channel_df['value'] = ICM_channel_df['value'].astype(int)
ICM_event_df['value'] = ICM_event_df['value'].astype(int)
ICM_genre_df['value'] = ICM_genre_df['value'].astype(int)
ICM_subgenre_df['value'] = ICM_subgenre_df['value'].astype(int)

In [ ]:
#Let's use properties of the sparse matrices
import scipy.sparse as sps
import numpy as np
from numpy import linalg as LA
URM_sparse = sps.coo_matrix((URM_df['interaction'].values,(URM_df['userID'].values,URM_df['itemID'].values)))
URM_csr = URM_sparse.tocsr()

In [ ]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_user_wise
from Evaluation.Evaluator import EvaluatorHoldout

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_csr , train_percentage = 0.80)
#URM_train, URM_validation = split_train_in_two_percentage_user_wise(URM_sparse.tocsr(), train_percentage = 0.8, verbose = False)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender_mod import SLIMElasticNetRecommender
import random 
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.BaseRecommender import BaseRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

In [ ]:
import pyximport
pyximport.install()

In [ ]:
#prepare the environment to run Cython code
!python run_compile_all_cython.py

In [ ]:
class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1/norm*alpha + R2/norm*(1-alpha) where R1 and R2 come from
    algorithms trained on different loss functions.

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
        
    def fit(self, norm, alpha = 0.5):

        self.alpha = alpha
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        if(self.recommender_1.RECOMMENDER_NAME == "DifferentLossScoresHybridRecommender"):
            item_weights_1 = self.recommender_1._compute_item_score(user_id_array,items_to_compute) 
        else:    
            item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
    
        if(self.recommender_2.RECOMMENDER_NAME == "DifferentLossScoresHybridRecommender"):
            item_weights_2 = self.recommender_2._compute_item_score(user_id_array,items_to_compute) 
        else:    
            item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * (1-self.alpha)

        return item_weights

In [ ]:
class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
    def fit(self, alpha = 0.5):
        self.alpha = alpha      


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        if(self.recommender_1.RECOMMENDER_NAME == "DifferentLossScoresHybridRecommender"):
            item_weights_1 = self.recommender_1._compute_item_score(user_id_array,items_to_compute) 
        else:    
            item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
    
        if(self.recommender_2.RECOMMENDER_NAME == "DifferentLossScoresHybridRecommender"):
            item_weights_2 = self.recommender_2._compute_item_score(user_id_array,items_to_compute) 
        else:    
            item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights

In [ ]:
slim_BPR = SLIM_BPR_Cython(URM_sparse)
slim_BPR.fit(topK=150, random_seed=1234, lambda_i=0.0055, lambda_j=0.0055, epochs=350, learning_rate=0.019927955775670296)

In [ ]:
iALS = IALSRecommender(URM_sparse)
iALS.fit(epochs = 157, num_factors = 54, alpha = 0.6754923563037951, reg = 0.0020435763517982174)

In [ ]:
RP3beta = RP3betaRecommender(URM_sparse)
RP3beta.fit(topK=120, alpha =  0.9308113538224418, beta = 0.32)

In [ ]:
itemKNN = ItemKNNCFRecommender(URM_sparse)
itemKNN.fit(topK=250, shrink=12)

In [ ]:
SLIM_tot = SLIMElasticNetRecommender(URM_sparse)
SLIM_tot.fit(l1_ratio = 0.0006874637222349307, alpha = 0.05475509828499467, positive_only = True, topK = 2822)

In [ ]:
cotrain1 = DifferentLossScoresHybridRecommender(URM_sparse, itemKNN, slim_BPR)
cotrain1.fit(alpha = 0.6144606244441579, norm = np.inf)

cotrain2 = DifferentLossScoresHybridRecommender(URM_sparse, iALS, RP3beta)
cotrain2.fit(alpha = 0.662714121265055, norm = np.inf)

In [ ]:
hybrid = DifferentLossScoresHybridRecommender(URM_sparse, cotrain1, cotrain2)
hybrid.fit(alpha = 0.7, norm = 2)

In [ ]:
final_hybrid = DifferentLossScoresHybridRecommender(URM_sparse, hybrid, SLIM_tot)
final_hybrid.fit(alpha = 0.35, norm = np.inf)